# Run fitsverify

In [1]:
import os
import sys
import re
import subprocess as sp
from configparser import ConfigParser
from random import choice
specprod = 'everest'
specprod_path = os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod)

## Create input file

In [2]:
fits_files = os.path.join(os.environ['CSCRATCH'], f'{specprod}_fits.txt')
if not os.path.exists(fits_files):
    # os.chdir(specprod_path)
    with open(fits_files, 'w') as out:
        command = ['find', 'calibnight', 'exposures', 'healpix', 'preproc', 'tiles', 'zcatalog', '-type', 'f', '-name', '*.fits']
        proc = sp.Popen(command, stdout=out, stderr=sp.DEVNULL, cwd=specprod_path)
        status = proc.wait()

## List of Regular Expressions

In [3]:
parser = ConfigParser()
parser.read_string("""
[top]
exposures = exposures-everest\.fits

[calibnight]
fiberflatnight = calibnight/[0-9]{8}/fiberflatnight-[brz][0-9]-[0-9]{8}\.fits
psfnight = calibnight/[0-9]{8}/psfnight-[brz][0-9]-[0-9]{8}\.fits

[exposures]
cframe = exposures/[0-9]{8}/[0-9]{8}/cframe-[brz][0-9]-[0-9]{8}\.fits
exposure-qa = exposures/[0-9]{8}/[0-9]{8}/exposure-qa-[0-9]{8}\.fits
fiberflat = exposures/[0-9]{8}/[0-9]{8}/fiberflat-[brz][0-9]-[0-9]{8}\.fits
fit-psf = exposures/[0-9]{8}/[0-9]{8}/fit-psf-[brz][0-9]-[0-9]{8}\.fits
fit-psf-before-blacklisted = exposures/[0-9]{8}/[0-9]{8}/fit-psf-before-blacklisted-[brz][0-9]-[0-9]{8}\.fits
fit-psf-before-blacklisted-fix = exposures/[0-9]{8}/[0-9]{8}/fit-psf-before-blacklisted-fix-[brz][0-9]-[0-9]{8}\.fits
fit-psf-fixed-blacklisted = exposures/[0-9]{8}/[0-9]{8}/fit-psf-fixed-blacklisted-[brz][0-9]-[0-9]{8}\.fits
fluxcalib = exposures/[0-9]{8}/[0-9]{8}/fluxcalib-[brz][0-9]-[0-9]{8}\.fits
frame = exposures/[0-9]{8}/[0-9]{8}/frame-[brz][0-9]-[0-9]{8}\.fits
psf = exposures/[0-9]{8}/[0-9]{8}/psf-[brz][0-9]-[0-9]{8}\.fits
sframe = exposures/[0-9]{8}/[0-9]{8}/sframe-[brz][0-9]-[0-9]{8}\.fits
shifted-input-psf = exposures/[0-9]{8}/[0-9]{8}/shifted-input-psf-[brz][0-9]-[0-9]{8}\.fits
sky = exposures/[0-9]{8}/[0-9]{8}/sky-[brz][0-9]-[0-9]{8}\.fits
stdstars = exposures/[0-9]{8}/[0-9]{8}/stdstars-[0-9]-[0-9]{8}\.fits

[healpix]
coadd = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/coadd-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits
qso_mgii = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/qso_mgii-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits
qso_qn = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/qso_qn-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits
redrock = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/redrock-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits
spectra = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/spectra-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits
tilepix = healpix/tilepix\.fits

[preproc]
fibermap = preproc/[0-9]{8}/[0-9]{8}/fibermap-[0-9]{8}\.fits
preproc = preproc/[0-9]{8}/[0-9]{8}/preproc-[brz][0-9]-[0-9]{8}\.fits

[tiles]
coadd = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/coadd-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits
qso_mgii = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/qso_mgii-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits
qso_qn = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/qso_qn-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits
redrock = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/redrock-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits
spectra = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/spectra-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits
tile-qa = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/tile-qa-[0-9]+-(thru|exp|)[0-9]{8}\.fits

[tiles:depth]
coadd = tiles/[14]x_depth/[0-9]+/[0-9]/coadd-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits
qso_mgii = tiles/[14]x_depth/[0-9]+/[0-9]/qso_mgii-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits
qso_qn = tiles/[14]x_depth/[0-9]+/[0-9]/qso_qn-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits
redrock = tiles/[14]x_depth/[0-9]+/[0-9]/redrock-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits
spectra = tiles/[14]x_depth/[0-9]+/[0-9]/spectra-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits

[zcatalog]
zpix = zcatalog/zpix-(sv1|sv2|sv3|main)-(backup|bright|dark|other)\.fits
ztile = zcatalog/ztile-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-(cumulative|pernight)\.fits

[exclude]
calibnight = calibnight/[0-9]{8}/tmp/*
exposures = exposures/[0-9]{8}/old/*
preproc = preproc/[0-9]{8}/old/*
"""
    )


## Precompile Regular Expressions

In [4]:
r = dict()
for s in parser.sections():
    r[s] = dict()
    for key, value in parser.items(s):
        r[s][key] = re.compile(value)

## Scan the list of files

In [5]:
with open(fits_files) as e:
    data = e.readlines()
data.append(f'exposures-{specprod}.fits\n')
scanable = dict()
for file in data:
    ff = file.strip()
    f = ff.split('/')
    if len(f) == 1:
        section = 'top'
    elif f[1] == '1x_depth' or f[1] == '4x_depth':
        section = 'tiles:depth'
    else:
        section = f[0]
    if section not in scanable:
        scanable[section] = dict()
    excluded = False
    for key in r['exclude']:
        m = r['exclude'][key].match(ff)
        if m is not None:
            excluded = True
    if excluded:
        continue
    matched = False
    for key in r[section]:
        m = r[section][key].match(ff)
        if m is not None:
            matched = True
            if key in scanable[section]:
                scanable[section][key].append(ff)
            else:
                scanable[section][key] = [ff]
    if matched:
        continue
    print("ERROR: Could not match {0}!".format(ff))

## From the list of all file types, pick one at random

In [6]:
scan = dict()
for section in scanable:
    scan[section] = dict()
    for ftype in scanable[section]:
        scan[section][ftype] = choice(scanable[section][ftype])
scan

{'calibnight': {'fiberflatnight': 'calibnight/20210201/fiberflatnight-b4-20210201.fits',
  'psfnight': 'calibnight/20210320/psfnight-r0-20210320.fits'},
 'exposures': {'frame': 'exposures/20210306/00079563/frame-r2-00079563.fits',
  'psf': 'exposures/20210521/00089682/psf-r2-00089682.fits',
  'fiberflat': 'exposures/20210606/00091359/fiberflat-r5-00091359.fits',
  'sky': 'exposures/20210629/00096687/sky-r8-00096687.fits',
  'sframe': 'exposures/20201220/00069051/sframe-r8-00069051.fits',
  'cframe': 'exposures/20210608/00092373/cframe-r7-00092373.fits',
  'fluxcalib': 'exposures/20210109/00071629/fluxcalib-z9-00071629.fits',
  'stdstars': 'exposures/20210202/00074292/stdstars-1-00074292.fits',
  'shifted-input-psf': 'exposures/20210316/00080628/shifted-input-psf-r2-00080628.fits',
  'fit-psf': 'exposures/20210506/00087422/fit-psf-r4-00087422.fits',
  'fit-psf-before-blacklisted-fix': 'exposures/20210615/00094726/fit-psf-before-blacklisted-fix-r5-00094726.fits',
  'fit-psf-fixed-blackli

## Run fitsverify on the chosen files

In [8]:
for section in scan:
    for key in scan[section]:
        command = ['fitsverify', '-l', scan[section][key]]
        proc = sp.Popen(command, stdout=sp.PIPE, stderr=sp.PIPE, cwd=specprod_path)
        out, err = proc.communicate()
        # print(section, key, out.decode('ascii').split('\n')[-2])
        result = out.decode('ascii').split('\n')[-2]
        if result == "**** Verification found 0 warning(s) and 0 error(s). ****":
            pass
            # print(section, key, "No problems detected.")
        else:
            print(section, key, "Problems detected!")
            print(out.decode('ascii'))
            if err:
                print(err.decode('ascii'))

calibnight fiberflatnight Problems detected!
 
              fitsverify 4.18 (CFITSIO V3.410)              
              --------------------------------              
 
 
File: calibnight/20210201/fiberflatnight-b4-20210201.fits

6 Header-Data Units in this file.
 
=================== HDU 1: Primary Array ===================
 
*** Warning: Keyword #195, FRAMES has a null value.
*** Warning: Keyword #206, OPENSHUT has a null value.
*** Warning: Keyword #36, EPOCH is deprecated. Use EQUINOX instead.
 
   1 | SIMPLE  =                    T / conforms to FITS standard
   2 | BITPIX  =                  -32 / array data type
   3 | NAXIS   =                    2 / number of array dimensions
   4 | NAXIS1  =                 2751
   5 | NAXIS2  =                  500
   6 | EXTEND  =                    T
   7 | EXTNAME = 'FIBERFLAT'
   8 | EXPID   =                74062 / Exposure number
   9 | EXPFRAME=                    0 / Frame number
  10 | FLAVOR  = 'science '           / Observation 